In [6]:
import geopandas as gpd
import geemap
import ee
import sys
import numpy as np
sys.path.append('..')
from scripts.data_processing_utils import DatasetMaker, DataLoader
import os
import cv2
ds_maker = DatasetMaker()
ds_loader = DataLoader()
geemap.ee_initialize()

import tensorflow as tf
import tensorflow.keras as keras
from models.networks.generator import dem_fill_net

generator = dem_fill_net()



# resolutions
scale = 30
large_dim = 256
small_dim = 64

# Set paths and dataset version
version = "V1.7.6"

# Set path for generator weights:
weights_path = f'/home/robin/Nextcloud_sn/Masterarbeit/Results/Cluster/{version}/save_gen/generator_final.keras'

generator.load_weights(weights_path)

# Glacier to evaluate
glac_path = '/home/robin/Nextcloud_sn/QGIS/processed/shapefiles/eval/trift_2003.gpkg'

# Create the directories
base_data_path = f'/home/robin/Nextcloud_sn/Masterarbeit/Results/Evaluation/{version}'
sub_paths = ['DEMs', 'Inner-Outer Mask', 'Intersection Mask', 'Intersection Mask Small', 'QGIS']

for sub_path in sub_paths:
    full_path = os.path.join(base_data_path, sub_path)
    os.makedirs(full_path, exist_ok=True)

    # Set the paths
file_path_dem = f'/home/robin/Nextcloud_sn/Masterarbeit/Results/Evaluation/{version}/DEMs'
path_inner_outer = f'/home/robin/Nextcloud_sn/Masterarbeit/Results/Evaluation/{version}/Inner-Outer Mask'
path_intersection = f'/home/robin/Nextcloud_sn/Masterarbeit/Results/Evaluation/{version}/Intersection Mask'
path_intersection_small = f'/home/robin/Nextcloud_sn/Masterarbeit/Results/Evaluation/{version}/Intersection Mask Small'

df = gpd.read_file(glac_path, crs = 'EPSG:3857')

glac_id = df['glac_id'].iloc[0]
file_name = str(glac_id) + '.tif'
file_name_mask = str(glac_id) + '.npy'

In [7]:
def load_dem_for_eval(file_path, resize = False, shape = (256, 256)):
    dem = cv2.imread(file_path, cv2.IMREAD_LOAD_GDAL)
    if resize:
        dem = cv2.resize(dem, dsize=shape, interpolation=cv2.INTER_AREA)

    min_val = np.min(dem)
    max_val = np.max(dem)

    image = tf.convert_to_tensor(dem)
    image = tf.expand_dims(image, axis=-1)
    image = tf.expand_dims(image, axis=0)

    return image, min_val, max_val

def load_mask_for_eval(file_path):
    mask = np.load(file_path)
    mask = tf.convert_to_tensor(mask)
    mask = tf.expand_dims(mask, axis=-1)
    mask = tf.expand_dims(mask, axis=0)

    return mask

In [8]:

gdf_bbox_rand, gdf_intersection = ds_maker.get_bbox_center(df, spatial_resolution=scale, img_res=large_dim)

if ds_maker.glacier_area_check(scale, small_dim, df):
    print('Area of glacier bounds is larger than area of small bbox')

    gdf_bbox_rand_small, gdf_intersection_small = ds_maker.get_random_bbox_intersect(gdf_intersection, spatial_resolution = scale,
                                                                                                 img_res = small_dim, buffer = 0, min_coverage = 0.01,
                                                                                                 max_tries = 1000)
    
    if gdf_bbox_rand_small is None or gdf_intersection_small is None:
        print('No intersection found')
        print('Not good!')
    else:
        # Generate masks
        inner_outer_mask, intersection_mask, intersection_mask_small = ds_maker.bbox_2_mask_rasterized(gdf_bbox_rand, gdf_bbox_rand_small, gdf_intersection, gdf_intersection_small,
                                                                                                        properties = {
                                                                                                'outer_dim' : (large_dim, large_dim),
                                                                                                'inner_dim' : (small_dim, small_dim),
                                                                                            })
else:
    gdf_bbox_rand_small, gdf_intersection_small = ds_maker.get_bbox_center(df, spatial_resolution=scale, img_res=small_dim)

    inner_outer_mask, intersection_mask, intersection_mask_small = ds_maker.bbox_2_mask_rasterized(gdf_bbox_rand, gdf_bbox_rand_small, gdf_intersection, gdf_intersection_small,
                                                                                                                   properties = {
                                                                                                            'outer_dim' : (large_dim, large_dim),
                                                                                                            'inner_dim' : (small_dim, small_dim),
                                                                                                        })
    
# Download and Save DEM
path_abs = os.path.join(file_path_dem, file_name)        
sq_geom = geemap.geopandas_to_ee(gdf_bbox_rand)
filtered_band = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('DEM').filterBounds(sq_geom).mean()
geemap.ee_export_image(filtered_band, filename=path_abs, scale=scale, region=sq_geom.geometry().bounds(), file_per_band=False, crs = 'EPSG:3857')

# Save masks
np.save(os.path.join(path_inner_outer, file_name_mask), inner_outer_mask)
np.save(os.path.join(path_intersection, file_name_mask), intersection_mask)
np.save(os.path.join(path_intersection_small, file_name_mask), intersection_mask_small)

Area of glacier bounds is larger than area of small bbox
Generating URL ...
Please wait ...
Data downloaded to /home/robin/Nextcloud_sn/Masterarbeit/Results/Evaluation/V1.7.6/DEMs/G007685E46051N.tif


In [9]:
#dem_path = ds_loader.populate_list(file_path_dem)
#inner_outer_mask = ds_loader.populate_list(path_inner_outer)
#intersection_mask = ds_loader.populate_list(path_intersection)
#intersection_mask_small = ds_loader.populate_list(path_intersection_small)

dem_path = os.path.join(file_path_dem, file_name)
inner_outer_mask = os.path.join(path_inner_outer, file_name_mask)
intersection_mask = os.path.join(path_intersection, file_name_mask)
intersection_mask_small = os.path.join(path_intersection_small, file_name_mask)

dem_org, min_val, max_val = load_dem_for_eval(dem_path, resize = True, shape = (256, 256))
inner_outer_mask = load_mask_for_eval(inner_outer_mask)
intersection_mask = load_mask_for_eval(intersection_mask)
intersection_mask_small = load_mask_for_eval(intersection_mask_small)

# Normalize the DEM [0, 1]
#dem = (dem_org - min_val) / (max_val - min_val)

# Normalize the DEM [-1, 1]
dem = ((dem_org - min_val) / (max_val - min_val)) *2 - 1


# Get output from generator and norm it back to org state. 
output_s1, output_s2 = generator([dem, inner_outer_mask, intersection_mask], training = False)
#dem_s2 = output_s2 * (max_val - min_val) + min_val

# From [-1, 1] to original
dem_s2 = ((output_s2 + 1) / 2) * (max_val - min_val) + min_val

# Remove batch dimension
dem_s2 = tf.squeeze(dem_s2, axis=0)

# Augment, so that only glacier area is shown

dem_s2 = dem_s2 * intersection_mask + dem_org * (1 - intersection_mask)





In [10]:
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, reproject

file_out = glac_id + '_s2.tif'

with rio.open(dem_path) as dem_source:
    dem_profile = dem_source.profile


with rio.open(os.path.join(file_path_dem, file_out), 'w', **dem_profile) as dest:
    dest.write(dem_s2[0, :, :, 0], 1)